In [16]:
import http.client
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
REDIS_URL = 'mbad1su6x4.execute-api.us-east-1.amazonaws.com'
MEMCACHE_URL = 'muka7ucb0a.execute-api.us-east-1.amazonaws.com'

In [17]:
def sendRequest(url, method, body):
  conn = http.client.HTTPSConnection(url)
  payload = json.dumps(body)
  headers = {
    'content-type': 'application/json'
  }
  conn.request("POST", f'/prod/{method}', payload, headers)
  res = conn.getresponse()
  return json.loads(res.read())
def resetRedis():
  sendRequest(REDIS_URL, 'reset-redis', {})
def resetMemcache():
  sendRequest(REDIS_URL, 'reset-memcached', {})

In [65]:
class TestCache:
    def __init__(self):
        url = ''
        chargeUrl = ''
        resetUrl = ''
    
    def testBasicCase(self):
        sendRequest(self.url, self.resetUrl, {})
        for i in range(120):
            res = sendRequest(self.url, self.chargeUrl, {'unit': 1})
            if i < 100:
                assert res['isAuthorized'] == True
                assert res['remainingBalance'] == 100-i-1
            else:
                if res['isAuthorized']:
                    assert res['isAuthorized'] == False
            assert res['remainingBalance'] >= 0
    def testConcurrentCase(self):
        sendRequest(self.url, self.resetUrl, {})
        futures = []
        count = 0
        with ThreadPoolExecutor() as e:
            for i in range(120):
                futures.append(e.submit(sendRequest, url=self.url, method = self.chargeUrl, body = {'unit': 1}))
            for f in as_completed(futures):
                if 'isAuthorized' in f.result() and f.result()['isAuthorized']:
                    count+=1
                if 'remainingBalance' in f.result():
                    assert f.result()['remainingBalance'] >= 0
        assert count == 100

In [68]:
class RedisTest(TestCache):
    def __init__(self):
        self.url = REDIS_URL
        self.chargeUrl = 'charge-request-redis'
        self.resetUrl = 'reset-redis'
class MemCacheTest(TestCache):
    def __init__(self):
        self.url = MEMCACHE_URL
        self.chargeUrl = 'charge-request-memcached'
        self.resetUrl = 'reset-memcached'

In [69]:
r = RedisTest()
r.testBasicCase()
r.testConcurrentCase()

In [ ]:
m = MemCacheTest()
m.testBasicCase()
m.testConcurrentCase()